In [2]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from deepface import DeepFace

In [3]:
load_dotenv("../.env")

True

In [6]:
import os
import requests
import pandas as pd
from deepface import DeepFace

X_API_KEY = os.getenv('X-API-KEY')  # Assuming the API key is in the environment variable
only_search_ai_endpoint = os.getenv('ONLY_SEARCH_AI_ENDPOINT')
num_pages = 500  # or however many pages you want to process
start_page = 250

def analyze_gender_from_image(image_url):
    try:
        print(f"Running DeepFace for img_url: {image_url}")
        result = DeepFace.analyze(image_url, actions=['gender'])
        gender = result[0]['dominant_gender']
        confidence = result[0]['gender']
        return {'gender': gender.lower(), 'confidence': confidence}
    except Exception as e:
        print("DeepFace Error:", e)
        return {'gender': 'unknown', 'confidence': {'Woman': 0, 'Man': 0}}

def run_profile_and_predict_gender(num_pages, profile_url, output_dir='../outputs',start_page=3):
    os.makedirs(output_dir, exist_ok=True)

    for page in range(start_page, num_pages + 1):
        print(f"\n=== Processing page {page} ===")
        gender_data = []

        try:
            response = requests.post(url=profile_url, json={"page": page})
            response.raise_for_status()
            data = response.json().get('data', [])
        except Exception as e:
            print(f"Failed to fetch profiles for page {page}: {e}")
            data = []  # Continue with an empty list so CSV still gets saved

        for item in data:
            try:
                profile = item.get('ProfileDatum', {})
                username = item.get('username', None)
                avatar = profile.get('avatar', None)

                if avatar:
                    print(f'username: {username} - avatar: {avatar}')
                    gender_response = analyze_gender_from_image(avatar)
                    gender = gender_response.get('gender', 'unknown')
                    confidence_score = gender_response.get('confidence', {'Woman': 0, 'Man': 0})
                else:
                    gender = 'unknown'
                    confidence_score = {'Woman': 0, 'Man': 0}

                gender_data.append({
                    'username': username,
                    'gender': gender,
                    'confidence_score': confidence_score,
                    'avatar': avatar
                })

            except Exception as e:
                print(f"Error processing profile for username {username}: {e}")
                continue  # Skip to next profile
        df = pd.DataFrame(gender_data)
        output_path = os.path.join(output_dir, f'user_attributes_{page}.csv')
        print('output_path',output_path)
        df.to_csv(output_path, index=False)
        print(f"Saved page {page} results to {output_path} (rows: {len(df)})")

In [7]:
profile_url = os.getenv('OF_PROFILES_URL')  # Ensure the URL is set in the environment
df = run_profile_and_predict_gender(num_pages, profile_url, output_dir='../outputs',start_page=250)


=== Processing page 250 ===
username: connycosplay - avatar: https://public.onlyfans.com/files/x/xo/xow/xowhnzhvf0mntnbmfjipypofo7nqv8ca1704676306/54413341/avatar.jpg
Running DeepFace for img_url: https://public.onlyfans.com/files/x/xo/xow/xowhnzhvf0mntnbmfjipypofo7nqv8ca1704676306/54413341/avatar.jpg
username: connyfox - avatar: https://public.onlyfans.com/files/i/io/iod/iodlwuhmxt48mm1phd6vklo9aiklgoje1639429368/avatar.jpg
Running DeepFace for img_url: https://public.onlyfans.com/files/i/io/iod/iodlwuhmxt48mm1phd6vklo9aiklgoje1639429368/avatar.jpg
DeepFace Error: Face could not be detected in https://public.onlyfans.com/files/i/io/iod/iodlwuhmxt48mm1phd6vklo9aiklgoje1639429368/avatar.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
username: conorhurhur - avatar: https://public.onlyfans.com/files/z/zw/zwb/zwbsjsgxb2p4c5benxkipaotracdl6f41729245728/28623194/avatar.jpg
Running DeepFace for img_url: https://public.onlyfans.com/fil

: 